# Overview

This colab demonstrates the steps to use the DeepLab model to perform semantic segmentation on a sample input image. Expected outputs are semantic labels overlayed on the sample image.

### About DeepLab
The models used in this colab perform semantic segmentation. Semantic segmentation models focus on assigning semantic labels, such as sky, person, or car, to multiple objects and stuff in a single image.

## Import Libraries

In [3]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

#%tensorflow_version 1.x
import tensorflow as tf


## Import helper methods
These methods help us perform the following tasks:
* Load the pretrained DeepLab model
* Load the colormap from the PASCAL VOC dataset
* Adds colors to various labels
* Visualize an image, and add an overlay of colors on various regions

In [4]:
from tensorflow.python.platform import gfile
class TinyYoloV3Model(object):
  """Class to load model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, model_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    file_handle = gfile.FastGFile(model_path,'rb')
    graph_def = tf.GraphDef.FromString(file_handle.read())

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)
  
  def entry_index(side, coord, classes, location, entry):
    side_power_2 = side ** 2
    n = location // side_power_2
    loc = location % side_power_2
    return int(side_power_2 * (n * (coord + classes + 1) + entry) + loc)

  def scale_bbox(x, y, h, w, class_id, confidence, h_scale, w_scale):
    xmin = int((x - w / 2) * w_scale)
    ymin = int((y - h / 2) * h_scale)
    xmax = int(xmin + w * w_scale)
    ymax = int(ymin + h * h_scale)
    return dict(xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, class_id=class_id, confidence=confidence)

  def intersection_over_union(box_1, box_2):
    width_of_overlap_area = min(box_1['xmax'], box_2['xmax']) - max(box_1['xmin'], box_2['xmin'])
    height_of_overlap_area = min(box_1['ymax'], box_2['ymax']) - max(box_1['ymin'], box_2['ymin'])
    if width_of_overlap_area < 0 or height_of_overlap_area < 0:
      area_of_overlap = 0
    else:
      area_of_overlap = width_of_overlap_area * height_of_overlap_area
    box_1_area = (box_1['ymax'] - box_1['ymin']) * (box_1['xmax'] - box_1['xmin'])
    box_2_area = (box_2['ymax'] - box_2['ymin']) * (box_2['xmax'] - box_2['xmin'])
    area_of_union = box_1_area + box_2_area - area_of_overlap
    if area_of_union == 0:
      return 0
    return area_of_overlap / area_of_union

  def parse_yolo_region(blob, resized_image_shape, original_im_shape, params, threshold):
    # ------------------------------------------ Validating output parameters ------------------------------------------
    _, _, out_blob_h, out_blob_w = blob.shape
    assert out_blob_w == out_blob_h, "Invalid size of output blob. It sould be in NCHW layout and height should " \
                     "be equal to width. Current height = {}, current width = {}" \
                     "".format(out_blob_h, out_blob_w)

    # ------------------------------------------ Extracting layer parameters -------------------------------------------
    orig_im_h, orig_im_w = original_im_shape
    resized_image_h, resized_image_w = resized_image_shape
    objects = list()
    predictions = blob.flatten()
    side_square = params.side * params.side

    # ------------------------------------------- Parsing YOLO Region output -------------------------------------------
    for i in range(side_square):
      row = i // params.side
      col = i % params.side
      for n in range(params.num):
        obj_index = TinyYOLOv3.entry_index(params.side, params.coords, params.classes, n * side_square + i, params.coords)
        scale = predictions[obj_index]
        if scale < threshold:
          continue
        box_index = TinyYOLOv3.entry_index(params.side, params.coords, params.classes, n * side_square + i, 0)
        x = (col + predictions[box_index + 0 * side_square]) / params.side * resized_image_w
        y = (row + predictions[box_index + 1 * side_square]) / params.side * resized_image_h
        # Value for exp is very big number in some cases so following construction is using here
        try:
          w_exp = exp(predictions[box_index + 2 * side_square])
          h_exp = exp(predictions[box_index + 3 * side_square])
        except OverflowError:
          continue
        w = w_exp * params.anchors[params.anchor_offset + 2 * n]
        h = h_exp * params.anchors[params.anchor_offset + 2 * n + 1]
        for j in range(params.classes):
          class_index = TinyYOLOv3.entry_index(params.side, params.coords, params.classes, n * side_square + i,
            params.coords + 1 + j)
          confidence = scale * predictions[class_index]
          if confidence < threshold:
            continue
          objects.append(TinyYOLOv3.scale_bbox(x=x, y=y, h=h, w=w, class_id=j, confidence=confidence,
            h_scale=orig_im_h / resized_image_h, w_scale=orig_im_w / resized_image_w))
    return objects    


  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'background', 'leaf', 'other', 'other', 'other', 'other', 'other',
    'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other',
    'other', 'other', 'other', 'other', 'other', 'other'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

## Select a pretrained model
Select the pretrained model.

In [5]:
MODEL = TinyYoloV3Model('pbmodel/yolov3-tiny-leaf-train_best.graph')
print('model loaded successfully!')

Instructions for updating:
Use tf.gfile.GFile.
model loaded successfully!


In [ ]:

def run_visualization(image_path):
  """Inferences DeepLab model and visualizes result."""
  original_im = Image.open(image_path)

  print('running deeplab on image %s...' % image_path)
  resized_im, seg_map = MODEL.run(original_im)
  print(seg_map[256][256])

  vis_segmentation(resized_im, seg_map)


with open("./data/valid.txt", "r") as f:
    testfile = f.readline()[:-1]
print(testfile)
run_visualization('{0}'.format(testfile))

### export model to tensorboard and start tensorboard

In [9]:
# with tf.Session() as sess:
    model_filename ="pbmodel/yolov3-tiny-leaf-train_best.pb"
    with gfile.FastGFile(model_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)

        sess = tf.Session(graph=g_in)
    LOGDIR="pbmodel"
    train_writer = tf.summary.FileWriter(LOGDIR)
    train_writer.add_graph(sess.graph)

In [10]:
%load_ext tensorboard

In [12]:
%tensorboard --logdir pbmodel